 # Database for genotyping data

In [2]:
# import libraries
import csv
import pandas as pd
import sqlite3

We need to create a database and insert data from two dataframes: the first one with genotyping data and the second one with metadata.

In [269]:
# create database
connection = sqlite3.connect('genotyping_data.db')

In [270]:
# read metadata
metadata = pd.read_csv('genotyping_data/metadata.csv', index_col=0)

We set ```dna_chip_id``` as PRIMARY KEY, because this column can serve as unique identifier of the samples. However, it's not common practice to use character variables as keys (integers are less memory consuming) 

In [271]:
# create a table for metadata in db with colnames and their types
query = '''CREATE TABLE metadata 
            ("dna_chip_id" TEXT PRIMARY KEY, 
            "breed" TEXT, 
            "sex" TEXT)'''
connection.execute(query)
connection.commit()

We can insert data from dataframe via ```to_sql``` function from pandas:

In [272]:
# insert df into db
metadata.to_sql('metadata', con=connection, if_exists='append', index=False, chunksize=10000)

In [273]:
# read genotyping data
genstudio = pd.read_csv('genotyping_data/genstudio.csv', index_col=0)

In [274]:
# rename Sample ID column
genstudio = genstudio.rename(columns={'Sample ID' : 'dna_chip_id'})

In [275]:
# how many SNPs and samples?
genstudio[['dna_chip_id']].value_counts()

dna_chip_id       
202341831114R08C01    68516
202341831127R03C01    68516
202341831114R01C02    68516
202341831114R02C01    68516
202341831114R02C02    68516
202341831114R03C01    68516
202341831114R03C02    68516
202341831114R04C01    68516
202341831114R04C02    68516
202341831114R05C01    68516
202341831114R05C02    68516
202341831114R06C01    68516
202341831114R06C02    68516
202341831114R07C01    68516
202341831114R07C02    68516
202341831114R01C01    68516
202341831114R08C02    68516
202341831114R09C01    68516
202341831114R09C02    68516
202341831114R10C01    68516
202341831114R10C02    68516
202341831114R11C01    68516
202341831114R11C02    68516
202341831114R12C01    68516
202341831114R12C02    68516
202341831127R01C01    68516
202341831127R01C02    68516
202341831127R02C01    68516
202341831127R02C02    68516
202341831127R03C02    13036
dtype: int64

68516 SNPs were genotyped in each of 30 sample, which means that there should be many-to-many link between our dataframes.

In terms of genotyping data We set ```genotyping_id``` as PRIMARY KEY, which is simply a row id. Also, we add FOREIGN KEY that links two tables by ```dna_chip_id``` column. Finally, ON DELETE CASCADE method was added. It means that deleting data from metadata, we will automatically delete rows from genotyping data with the same ```dna_chip_id```.

It is very helpful, if we want to delete all SNPs that were genotyped for exactly one sample.

In [276]:
# create new table for genotyping data in db with colnames and their types
query = '''CREATE TABLE genstudio 
            ("genotyping_id" INTEGER PRIMARY KEY, 
            "SNP Name" TEXT, 
            "SNP Index" INTEGER, 
            "SNP Aux" INTEGER, 
            "dna_chip_id" TEXT, 
            "SNP" TEXT, 
            "Allele1 - Top" TEXT, 
            "Allele2 - Top" TEXT, 
            "Allele1 - Forward" TEXT, 
            "Allele2 - Forward" TEXT, 
            "Allele1 - AB" TEXT, 
            "Allele2 - AB" TEXT, 
            "Chr" TEXT, 
            "Position" TEXT, 
            "GC Score" REAL, 
            "GT Score" REAL, 
            "Theta" REAL, 
            "R" REAL, 
            "B Allele Freq" REAL, 
            "Log R Ratio" REAL,
            CONSTRAINT fk_metadata
                FOREIGN KEY ("dna_chip_id")
                REFERENCES metadata("dna_chip_id")
                ON DELETE CASCADE)'''
connection.execute(query)
connection.commit()

In [277]:
# insert df into db
genstudio.to_sql('genstudio', con=connection, if_exists='append', index=False, chunksize=10000)

Our database is ready. Now we can fetch some data from both tables using JOIN method.

In [278]:
# query only 5 first rows, all columns, joined by sample id:
query = '''
    SELECT * FROM genstudio
    JOIN metadata ON metadata."dna_chip_id" = genstudio."dna_chip_id"
    LIMIT 5'''
res = connection.execute(query)

In [279]:
for row in res.fetchall():
    print(row)

(1, '1_10573221', 1, 0, '202341831114R01C01', '[T/C]', '-', '-', '-', '-', '-', '-', '1', '10573221', 0.0, 0.0, 0.9420000000000001, 0.413, 1.0, 0.404, '202341831114R01C01', 'Д', 'Хр')
(2, '1_10673082', 2, 0, '202341831114R01C01', '[T/C]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '10673082', 0.8272, 0.8076, 0.039, 0.968, 0.0, 0.3017, '202341831114R01C01', 'Д', 'Хр')
(3, '1_10723065', 3, 0, '202341831114R01C01', '[A/G]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '10723065', 0.8316, 0.8107, 0.011000000000000001, 1.577, 0.0, 0.0388, '202341831114R01C01', 'Д', 'Хр')
(4, '1_11337555', 4, 0, '202341831114R01C01', '[A/G]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '11337555', 0.3781, 0.7925, 0.045, 1.104, 0.0, 0.2761, '202341831114R01C01', 'Д', 'Хр')
(5, '1_11407894', 5, 0, '202341831114R01C01', '[A/G]', 'G', 'G', 'G', 'G', 'B', 'B', '1', '11407894', 0.9038, 0.867, 0.983, 1.122, 0.9994, 0.0022, '202341831114R01C01', 'Д', 'Хр')


## UPDATE method

As you can see, ```SNP Name``` is coded as chr_position. However some bioinformatics tools accept SNP names in "chr:position" style (with colon). So we need to update ```SNP Name```. Let's try for the chromosome 1 only.

In [280]:
query = '''
    UPDATE genstudio
    SET "SNP Name" = Chr || ":" || Position
    WHERE CHR = 1'''
connection.execute(query)
connection.commit()

In [281]:
# query only 5 first rows, all columns, joined by sample id:
query = '''
    SELECT "SNP Name" FROM genstudio
    LIMIT 3'''
res = connection.execute(query)

In [282]:
for row in res.fetchall():
    print(row)

('1:10573221',)
('1:10673082',)
('1:10723065',)


Now ```SNP Name``` is updated!

## CASCADE DELETE method

Say we found out that the DNA sample 202341831114R02C01 was ruined and should be excluded from the analysis.

ON CASCADE DELETE comes in handy!

In [283]:
query = '''
    PRAGMA foreign_keys=on'''
connection.execute(query)
connection.commit()

We deleted a single row from metadata table, and all the rows with that DNA id were automatically deleted from the genstudio table.

In [284]:
query = '''
    DELETE FROM metadata
    WHERE "dna_chip_id" = "202341831114R02C01"'''
connection.execute(query)
connection.commit()

Let's check that no SNPs genotyped for that altered sample remained in genstudio table:

In [285]:
query = '''
    SELECT * FROM genstudio
    WHERE "dna_chip_id" = "202341831114R02C01"
    LIMIT 3'''
res = connection.execute(query)
for row in res.fetchall():
    print(row)

## DELETE method

Suppose we are not interested in rare variants (allele frequency < 0.01). We can simply delete such SNPs:

In [286]:
query = '''
    DELETE FROM genstudio
    WHERE "B Allele Freq" < 0.01'''
connection.execute(query)
connection.commit()

**Note** that ```dna_chip_id``` of rare variants were not delted from metadata table, which means that ON CASCADE DELETE doesn't work vice versa. 

In [287]:
# close database
connection.close()